## 1. 패키지/모듈 임포트

In [ ]:
import asyncio
import time
from indy import anoncreds, crypto, did, ledger, pool, wallet

import json
from typing import Optional

## 2. 함수 선언

In [ ]:
async def onboarding(_from, to):
    print("\"{}\" -> Create and store in Wallet \"{} {}\" DID".format(_from['name'], _from['name'], to['name']))
    (from_to_did, from_to_key) = await did.create_and_store_my_did(_from['wallet'], "{}")

    print("\"{}\" -> Send Nym to Ledger for \"{} {}\" DID".format(_from['name'], _from['name'], to['name']))
    await send_nym(_from['pool'], _from['wallet'], _from['did'], from_to_did, from_to_key, None)

    print("\"{}\" -> Send connection request to {} with \"{} {}\" DID and nonce"
                .format(_from['name'], to['name'], _from['name'], to['name']))
    connection_request = {
        'did': from_to_did,
        'nonce': 123456789
    }

    if 'wallet' not in to:
        print("\"{}\" -> Create wallet".format(to['name']))
        try:
            await wallet.create_wallet(to['wallet_config'], to['wallet_credentials'])
        except IndyError as ex:
            if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
                pass
        to['wallet'] = await wallet.open_wallet(to['wallet_config'], to['wallet_credentials'])

    print("\"{}\" -> Create and store in Wallet \"{} {}\" DID".format(to['name'], to['name'], _from['name']))
    (to_from_did, to_from_key) = await did.create_and_store_my_did(to['wallet'], "{}")

    print("\"{}\" -> Get key for did from \"{}\" connection request".format(to['name'], _from['name']))
    from_to_verkey = await did.key_for_did(_from['pool'], to['wallet'], connection_request['did'])

    print("\"{}\" -> Anoncrypt connection response for \"{}\" with \"{} {}\" DID, verkey and nonce"
                .format(to['name'], _from['name'], to['name'], _from['name']))
    to['connection_response'] = json.dumps({
        'did': to_from_did,
        'verkey': to_from_key,
        'nonce': connection_request['nonce']
    })
    to['anoncrypted_connection_response'] = \
        await crypto.anon_crypt(from_to_verkey, to['connection_response'].encode('utf-8'))

    print("\"{}\" -> Send anoncrypted connection response to \"{}\"".format(to['name'], _from['name']))
    _from['anoncrypted_connection_response'] = to['anoncrypted_connection_response']

    print("\"{}\" -> Anondecrypt connection response from \"{}\"".format(_from['name'], to['name']))
    _from['connection_response'] = \
        json.loads((await crypto.anon_decrypt(_from['wallet'], from_to_key,
                                              _from['anoncrypted_connection_response'])).decode("utf-8"))

    print("\"{}\" -> Authenticates \"{}\" by comparision of Nonce".format(_from['name'], to['name']))
    assert connection_request['nonce'] == _from['connection_response']['nonce']

    print("\"{}\" -> Send Nym to Ledger for \"{} {}\" DID".format(_from['name'], to['name'], _from['name']))
    await send_nym(_from['pool'], _from['wallet'], _from['did'], to_from_did, to_from_key, None)

    return from_to_did, from_to_key, to_from_did, to_from_key, _from['connection_response']


async def get_verinym(_from, from_to_did, from_to_key, to, to_from_did, to_from_key):
    print("\"{}\" -> Create and store in Wallet \"{}\" new DID".format(to['name'], to['name']))
    (to_did, to_key) = await did.create_and_store_my_did(to['wallet'], "{}")

    print("\"{}\" -> Authcrypt \"{} DID info\" for \"{}\"".format(to['name'], to['name'], _from['name']))
    to['did_info'] = json.dumps({
        'did': to_did,
        'verkey': to_key
    })
    to['authcrypted_did_info'] = \
        await crypto.auth_crypt(to['wallet'], to_from_key, from_to_key, to['did_info'].encode('utf-8'))

    print("\"{}\" -> Send authcrypted \"{} DID info\" to {}".format(to['name'], to['name'], _from['name']))

    print("\"{}\" -> Authdecrypted \"{} DID info\" from {}".format(_from['name'], to['name'], to['name']))
    sender_verkey, authdecrypted_did_info_json, authdecrypted_did_info = \
        await auth_decrypt(_from['wallet'], from_to_key, to['authcrypted_did_info'])

    print("\"{}\" -> Authenticate {} by comparision of Verkeys".format(_from['name'], to['name'], ))
    assert sender_verkey == await did.key_for_did(_from['pool'], _from['wallet'], to_from_did)

    print("\"{}\" -> Send Nym to Ledger for \"{} DID\" with {} Role"
                .format(_from['name'], to['name'], to['role']))
    await send_nym(_from['pool'], _from['wallet'], _from['did'], authdecrypted_did_info['did'],
                   authdecrypted_did_info['verkey'], to['role'])

    return to_did


async def send_nym(pool_handle, wallet_handle, _did, new_did, new_key, role):
    nym_request = await ledger.build_nym_request(_did, new_did, new_key, None, role)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, nym_request)


async def send_schema(pool_handle, wallet_handle, _did, schema):
    schema_request = await ledger.build_schema_request(_did, schema)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, schema_request)


async def send_cred_def(pool_handle, wallet_handle, _did, cred_def_json):
    cred_def_request = await ledger.build_cred_def_request(_did, cred_def_json)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, cred_def_request)


async def get_schema(pool_handle, _did, schema_id):
    get_schema_request = await ledger.build_get_schema_request(_did, schema_id)
    get_schema_response = await ledger.submit_request(pool_handle, get_schema_request)
    return await ledger.parse_get_schema_response(get_schema_response)


async def get_cred_def(pool_handle, _did, cred_def_id):
    get_cred_def_request = await ledger.build_get_cred_def_request(_did, cred_def_id)
    get_cred_def_response = await ledger.submit_request(pool_handle, get_cred_def_request)
    return await ledger.parse_get_cred_def_response(get_cred_def_response)


async def get_credential_for_referent(search_handle, referent):
    credentials = json.loads(
        await anoncreds.prover_fetch_credentials_for_proof_req(search_handle, referent, 10))
    return credentials[0]['cred_info']


async def prover_get_entities_from_ledger(pool_handle, _did, identifiers, actor):
    schemas = {}
    cred_defs = {}
    rev_states = {}
    for item in identifiers.values():
        print("\"{}\" -> Get Schema from Ledger".format(actor))
        (received_schema_id, received_schema) = await get_schema(pool_handle, _did, item['schema_id'])
        schemas[received_schema_id] = json.loads(received_schema)

        print("\"{}\" -> Get Credential Definition from Ledger".format(actor))
        (received_cred_def_id, received_cred_def) = await get_cred_def(pool_handle, _did, item['cred_def_id'])
        cred_defs[received_cred_def_id] = json.loads(received_cred_def)

        if 'rev_reg_seq_no' in item:
            pass  # TODO Create Revocation States

    return json.dumps(schemas), json.dumps(cred_defs), json.dumps(rev_states)


async def verifier_get_entities_from_ledger(pool_handle, _did, identifiers, actor):
    schemas = {}
    cred_defs = {}
    rev_reg_defs = {}
    rev_regs = {}
    for item in identifiers:
        print("\"{}\" -> Get Schema from Ledger".format(actor))
        (received_schema_id, received_schema) = await get_schema(pool_handle, _did, item['schema_id'])
        schemas[received_schema_id] = json.loads(received_schema)

        print("\"{}\" -> Get Credential Definition from Ledger".format(actor))
        (received_cred_def_id, received_cred_def) = await get_cred_def(pool_handle, _did, item['cred_def_id'])
        cred_defs[received_cred_def_id] = json.loads(received_cred_def)

        if 'rev_reg_seq_no' in item:
            pass  # TODO Get Revocation Definitions and Revocation Registries

    return json.dumps(schemas), json.dumps(cred_defs), json.dumps(rev_reg_defs), json.dumps(rev_regs)


async def auth_decrypt(wallet_handle, key, message):
    from_verkey, decrypted_message_json = await crypto.auth_decrypt(wallet_handle, key, message)
    decrypted_message_json = decrypted_message_json.decode("utf-8")
    decrypted_message = json.loads(decrypted_message_json)
    return from_verkey, decrypted_message_json, decrypted_message


# 3. 초기화, 시작~

In [ ]:
print("Getting started -> started")

# Set protocol version 2 to work with Indy Node 1.4
await pool.set_protocol_version(2)

pool_ = {
    'name': 'pool1',
    'config': json.dumps({"genesis_txn": '/home/indy/sandbox/pool_transactions_genesis'})
}
print("Open Pool Ledger: {}".format(pool_['name']))

try:
    await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
except IndyError as ex:
    if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
        pass

pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)

### ※ pool_transactions_genesis
<code>
{"reqSignature":{},"txn":{"data":{"data":{"alias":"Node1","blskey":"4N8aUNHSgjQVgkpm8nhNEfDf6txHznoYREg9kirmJrkivgL4oSEimFF6nsQ6M41QvhM2Z33nves5vfSn9n1UwNFJBYtWVnHYMATn76vLuL3zU88KyeAYcHfsih3He6UHcXDxcaecHVz6jhCYz1P2UZn2bDVruL5wXpehgBfBaLKm3Ba","blskey_pop":"RahHYiCvoNCtPTrVtP7nMC5eTYrsUA8WjXbdhNc8debh1agE9bGiJxWBXYNFbnJXoXhWFMvyqhqhRoq737YQemH5ik9oL7R4NTTCz2LEZhkgLJzB3QRQqJyBNyv7acbdHrAT8nQ9UkLbaVL9NBpnWXBTw4LEMePaSHEw66RzPNdAX1","client_ip":"10.0.0.2","client_port":9702,"node_ip":"10.0.0.2","node_port":9701,"services":["VALIDATOR"]},"dest":"Gw6pDLhcBcoQesN72qfotTgFa7cbuqZpkX3Xo6pLhPhv"},"metadata":{"from":"Th7MpTaRZVRYnPiabds81Y"},"type":"0"},"txnMetadata":{"seqNo":1,"txnId":"fea82e10e894419fe2bea7d96296a6d46f50f93f9eeda954ec461b2ed2950b62"},"ver":"1"}
{"reqSignature":{},"txn":{"data":{"data":{"alias":"Node2","blskey":"37rAPpXVoxzKhz7d9gkUe52XuXryuLXoM6P6LbWDB7LSbG62Lsb33sfG7zqS8TK1MXwuCHj1FKNzVpsnafmqLG1vXN88rt38mNFs9TENzm4QHdBzsvCuoBnPH7rpYYDo9DZNJePaDvRvqJKByCabubJz3XXKbEeshzpz4Ma5QYpJqjk","blskey_pop":"Qr658mWZ2YC8JXGXwMDQTzuZCWF7NK9EwxphGmcBvCh6ybUuLxbG65nsX4JvD4SPNtkJ2w9ug1yLTj6fgmuDg41TgECXjLCij3RMsV8CwewBVgVN67wsA45DFWvqvLtu4rjNnE9JbdFTc1Z4WCPA3Xan44K1HoHAq9EVeaRYs8zoF5","client_ip":"10.0.0.2","client_port":9704,"node_ip":"10.0.0.2","node_port":9703,"services":["VALIDATOR"]},"dest":"8ECVSk179mjsjKRLWiQtssMLgp6EPhWXtaYyStWPSGAb"},"metadata":{"from":"EbP4aYNeTHL6q385GuVpRV"},"type":"0"},"txnMetadata":{"seqNo":2,"txnId":"1ac8aece2a18ced660fef8694b61aac3af08ba875ce3026a160acbc3a3af35fc"},"ver":"1"}
{"reqSignature":{},"txn":{"data":{"data":{"alias":"Node3","blskey":"3WFpdbg7C5cnLYZwFZevJqhubkFALBfCBBok15GdrKMUhUjGsk3jV6QKj6MZgEubF7oqCafxNdkm7eswgA4sdKTRc82tLGzZBd6vNqU8dupzup6uYUf32KTHTPQbuUM8Yk4QFXjEf2Usu2TJcNkdgpyeUSX42u5LqdDDpNSWUK5deC5","blskey_pop":"QwDeb2CkNSx6r8QC8vGQK3GRv7Yndn84TGNijX8YXHPiagXajyfTjoR87rXUu4G4QLk2cF8NNyqWiYMus1623dELWwx57rLCFqGh7N4ZRbGDRP4fnVcaKg1BcUxQ866Ven4gw8y4N56S5HzxXNBZtLYmhGHvDtk6PFkFwCvxYrNYjh","client_ip":"10.0.0.2","client_port":9706,"node_ip":"10.0.0.2","node_port":9705,"services":["VALIDATOR"]},"dest":"DKVxG2fXXTU8yT5N7hGEbXB3dfdAnYv1JczDUHpmDxya"},"metadata":{"from":"4cU41vWW82ArfxJxHkzXPG"},"type":"0"},"txnMetadata":{"seqNo":3,"txnId":"7e9f355dffa78ed24668f0e0e369fd8c224076571c51e2ea8be5f26479edebe4"},"ver":"1"}
{"reqSignature":{},"txn":{"data":{"data":{"alias":"Node4","blskey":"2zN3bHM1m4rLz54MJHYSwvqzPchYp8jkHswveCLAEJVcX6Mm1wHQD1SkPYMzUDTZvWvhuE6VNAkK3KxVeEmsanSmvjVkReDeBEMxeDaayjcZjFGPydyey1qxBHmTvAnBKoPydvuTAqx5f7YNNRAdeLmUi99gERUU7TD8KfAa6MpQ9bw","blskey_pop":"RPLagxaR5xdimFzwmzYnz4ZhWtYQEj8iR5ZU53T2gitPCyCHQneUn2Huc4oeLd2B2HzkGnjAff4hWTJT6C7qHYB1Mv2wU5iHHGFWkhnTX9WsEAbunJCV2qcaXScKj4tTfvdDKfLiVuU2av6hbsMztirRze7LvYBkRHV3tGwyCptsrP","client_ip":"10.0.0.2","client_port":9708,"node_ip":"10.0.0.2","node_port":9707,"services":["VALIDATOR"]},"dest":"4PS3EDQ3dW1tci1Bp6543CfuuebjFrg36kLAUcskGfaA"},"metadata":{"from":"TWwCRQRZ2ZHMJFn9TzLp7W"},"type":"0"},"txnMetadata":{"seqNo":4,"txnId":"aa5e817d7cc626170eca175822029339a444eb0ee8f0bd20d3b0b76e566fb008"},"ver":"1"}
</code>

# 4. Wallet DID 생성
![Alt text](https://miro.medium.com/max/1098/0*-3m1wiCsA7pKqX1z "Steward creates the actors with Trust Anchor Role")
<center>Steward creates the actors with Trust Anchor Role</center>

## 4.1 Steward Wallet, DID 생성 by Steward

In [ ]:
print("\"Sovrin Steward\" -> Create wallet")
steward = {
    'name': "Sovrin Steward",
    'wallet_config': json.dumps({'id': 'sovrin_steward_wallet'}),
    'wallet_credentials': json.dumps({'key': 'steward_wallet_key'}),
    'pool': pool_['handle'],
    'seed': '000000000000000000000000Steward2'
}

try:
    await wallet.create_wallet(steward['wallet_config'], steward['wallet_credentials'])
except IndyError as ex:
    if ex.error_code == ErrorCode.WalletAlreadyExistsError:
        pass

steward['wallet'] = await wallet.open_wallet(steward['wallet_config'], steward['wallet_credentials'])

print("\"Sovrin Steward\" -> Create and store in Wallet DID from seed")
steward['did_info'] = json.dumps({'seed': steward['seed']})
steward['did'], steward['key'] = await did.create_and_store_my_did(steward['wallet'], steward['did_info'])

In [ ]:
async def create_wallet(to):
    print("\"{}\" -> Create wallet".format(to['name']))
    try:
        await wallet.create_wallet(to['wallet_config'], to['wallet_credentials'])
    except IndyError as ex:
        if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
            pass
    to['wallet'] = await wallet.open_wallet(to['wallet_config'], to['wallet_credentials'])

async def create_did(to):
    print("\"{}\" -> Create and store in Wallet \"{}\" new DID".format(to['name'], to['name']))
    (to_did, to_key) = await did.create_and_store_my_did(to['wallet'], "{}")

    to['did_info'] = json.dumps({
        'did': to_did,
        'verkey': to_key
    })
    
    return json.loads(to['did_info'])

async def get_verinym2(_from, to_name, to_role, to_did, to_key):
    print("\"{}\" -> Send Nym to Ledger for \"{} DID\" with {} Role"
                .format(_from['name'], to_name, to_role))
    await send_nym(_from['pool'], _from['wallet'], _from['did'], to_did,
                   to_key, to_role)
    
    return to_did

## 4.2 Government Wallet, DID 생성 by Goverment

In [ ]:
government = {
    'name': 'Government',
    'wallet_config': json.dumps({'id': 'government_wallet'}),
    'wallet_credentials': json.dumps({'key': 'government_wallet_key'}),
    'pool': pool_['handle'],
    'role': 'TRUST_ANCHOR'
}

# 지갑 생성
await create_wallet(government)

# DID 생성
government_did_info = await create_did(government)
print(government_did_info)

## 4.3 Government NYM(Identity Record) Ledger 저장 by Steward

In [ ]:
# Submitter, Name, Role(Trust Anchor), DID, Verkey
government['did'] = await get_verinym2(steward, government['name'], government['role'], government_did_info['did'], government_did_info['verkey'])

## 4.4 Faber College Wallet, DID 생성 by Faber College

In [ ]:
faber = {
        'name': 'Faber',
        'wallet_config': json.dumps({'id': 'faber_wallet'}),
        'wallet_credentials': json.dumps({'key': 'faber_wallet_key'}),
        'pool': pool_['handle'],
        'role': 'TRUST_ANCHOR'
}

# 지갑 생성
await create_wallet(faber)

# DID 생성
faber_did_info = await create_did(faber)
print(faber_did_info)

## 4.5 Faber College NYM(Identity Record) Ledger 저장 by Steward

In [ ]:
# Submitter, Name, Role(Trust Anchor), DID, Verkey
faber['did'] = await get_verinym2(steward, faber['name'], faber['role'], faber_did_info['did'], faber_did_info['verkey'])

## 4.6 Acme Corp Wallet, DID 생성 by Acme Corp

In [ ]:
acme = {
        'name': 'Acme',
        'wallet_config': json.dumps({'id': 'acme_wallet'}),
        'wallet_credentials': json.dumps({'key': 'acme_wallet_key'}),
        'pool': pool_['handle'],
        'role': 'TRUST_ANCHOR'
}

# 지갑 생성
await create_wallet(acme)

# DID 생성
acme_did_info = await create_did(acme)
print(acme_did_info)

## 4.7 Acme Corp NYM(Identity Record) Ledger 저장 by Steward

In [ ]:
# Submitter, Name, Role(Trust Anchor), DID, Verkey
acme['did'] = await get_verinym2(steward, acme['name'], acme['role'], acme_did_info['did'], acme_did_info['verkey'])

## 4.8 Thrift Bank Wallet, DID 생성 by Thrift Bank

In [ ]:
thrift = {
    'name': 'Thrift',
    'wallet_config': json.dumps({'id': 'thrift_wallet'}),
    'wallet_credentials': json.dumps({'key': 'thrift_wallet_key'}),
    'pool': pool_['handle'],
    'role': 'TRUST_ANCHOR'
}

# 지갑 생성
await create_wallet(thrift)

# DID 생성
thrift_did_info = await create_did(thrift)
print(thrift_did_info)

## 4.9 Thrift Bank NYM(Identity Record) Ledger 저장 by Steward

In [ ]:
# Submitter, Name, Role(Trust Anchor), DID, Verkey
thrift['did'] = await get_verinym2(steward, thrift['name'], thrift['role'], thrift_did_info['did'], thrift_did_info['verkey'])

# 5. Credential Schema 생성 - Transcript, Job Certificate by Government
![Alt text](https://miro.medium.com/max/1046/0*2RFOVKXlhGLa_1Gw "Government creates schemas")
<center>Government creates schemas</center>

In [ ]:
print("\"Government\" -> Create \"Transcript\" Schema")
# Transcript 스키마
transcript = {
    'name': 'Transcript',
    'version': '1.2',
    'attributes': ['first_name', 'last_name', 'degree', 'status', 'year', 'average', 'ssn']
}

(government['transcript_schema_id'], government['transcript_schema']) = \
    await anoncreds.issuer_create_schema(government['did'], transcript['name'], transcript['version'],
                                         json.dumps(transcript['attributes']))
transcript_schema_id = government['transcript_schema_id']

print("\"Government\" -> Send \"Transcript\" Schema to Ledger")
await send_schema(government['pool'], government['wallet'], government['did'], government['transcript_schema'])

In [ ]:
print("\"Government\" -> Create \"Job-Certificate\" Schema")
# Job Certificate 스키마
job_certificate = {
    'name': 'Job-Certificate',
    'version': '0.2',
    'attributes': ['first_name', 'last_name', 'salary', 'employee_status', 'experience']
}

(government['job_certificate_schema_id'], government['job_certificate_schema']) = \
  await anoncreds.issuer_create_schema(government['did'], job_certificate['name'], job_certificate['version'],
                                         json.dumps(job_certificate['attributes']))
job_certificate_schema_id = government['job_certificate_schema_id']

print("\"Government\" -> Send \"Job-Certificate\" Schema to Ledger")
await send_schema(government['pool'], government['wallet'], government['did'], government['job_certificate_schema'])

# 6 Credential Definition 생성 - Faber/Acme Credential Definition
![Alt text](https://miro.medium.com/max/1046/0*Gqf9nECRvcdxhY_d "Faber College and Acme Corp create credential definition based on schemas issued by Government")
<center>Faber College and Acme Corp create credential definition based on schemas issued by Government</center><br>

![Alt text](https://miro.medium.com/max/934/0*KI2GjNY0UOlYYORW "Relation between Schemas and Credential Definitions")
<center>Relation between Schemas and Credential Definitions</center>

In [ ]:
print("\"Faber\" -> Get \"Transcript\" Schema from Ledger")
(faber['transcript_schema_id'], faber['transcript_schema']) = \
    await get_schema(faber['pool'], faber['did'], transcript_schema_id)

print("\"Faber\" -> Create and store in Wallet \"Faber Transcript\" Credential Definition")
# Transcript - Schema + the necessary info(Issuer, Signature Type, Tag, Revocation)
transcript_cred_def = {
    'tag': 'TAG1',
    'type': 'CL',
    'config': {"support_revocation": False}
}
(faber['transcript_cred_def_id'], faber['transcript_cred_def']) = \
    await anoncreds.issuer_create_and_store_credential_def(faber['wallet'], faber['did'],
                                                           faber['transcript_schema'], transcript_cred_def['tag'],
                                                           transcript_cred_def['type'],
                                                           json.dumps(transcript_cred_def['config']))

print("\"Faber\" -> Send  \"Faber Transcript\" Credential Definition to Ledger")
await send_cred_def(faber['pool'], faber['wallet'], faber['did'], faber['transcript_cred_def'])

In [ ]:
print("\"Acme\" -> Get from Ledger \"Job-Certificate\" Schema")
(acme['job_certificate_schema_id'], acme['job_certificate_schema']) = \
    await get_schema(acme['pool'], acme['did'], job_certificate_schema_id)

print("\"Acme\" -> Create and store in Wallet \"Acme Job-Certificate\" Credential Definition")
# Transcript - Schema + the necessary info(Issuer, Signature Type, Tag, Revocation)
job_certificate_cred_def = {
    'tag': 'TAG1',
    'type': 'CL',
    'config': {"support_revocation": False}
}
(acme['job_certificate_cred_def_id'], acme['job_certificate_cred_def']) = \
    await anoncreds.issuer_create_and_store_credential_def(acme['wallet'], acme['did'],
                                                           acme['job_certificate_schema'],
                                                           job_certificate_cred_def['tag'],
                                                           job_certificate_cred_def['type'],
                                                           json.dumps(job_certificate_cred_def['config']))

print("\"Acme\" -> Send \"Acme Job-Certificate\" Credential Definition to Ledger")
await send_cred_def(acme['pool'], acme['wallet'], acme['did'], acme['job_certificate_cred_def'])

# 7. Faber College로 부터 Transcript 발급(for Alice)
![Alt text](https://miro.medium.com/max/1288/0*g7rQdxRt00XkySH6 "Alice gets credential from Faber College")
<center>Alice gets credential from Faber College</center><br>

## 7.1 Alice Wallet DID 생성 by Alice

In [ ]:
alice = {
        'name': 'Alice',
        'wallet_config': json.dumps({'id': 'alice_wallet'}),
        'wallet_credentials': json.dumps({'key': 'alice_wallet_key'}),
        'pool': pool_['handle'],
    }

# 지갑 생성
await create_wallet(alice)

# DID 생성
alice_did_info = await create_did(alice)
print(alice_did_info)

## 7.2 Alice NYM(Identity Record) Ledger 저장 by Faber College

In [ ]:
print("\"{}\" -> Send Nym to Ledger for \"{} {}\" DID".format(faber['name'], alice['name'], faber['name']))
await send_nym(faber['pool'], faber['wallet'], faber['did'], alice_did_info['did'], alice_did_info['verkey'], None)

## 7.3 [Faber -> Alice] Credential Offer 생성 및 전달

In [ ]:
print("\"Faber\" -> Create \"Transcript\" Credential Offer for Alice")

faber['transcript_cred_offer'] = await anoncreds.issuer_create_credential_offer(faber['wallet'], faber['transcript_cred_def_id'])
transcript_cred_offer = json.loads(faber['transcript_cred_offer'])

print("\"Faber\" -> Get key for Alice did")
faber['alic_key_for_faber'] = await did.key_for_did(faber['pool'], faber['wallet'], alice_did_info['did'])

print("\"Faber\" -> Send \"Transcript\" Credential Offer to Alice")
alice['transcript_schema_id'] = transcript_cred_offer['schema_id']
alice['transcript_cred_def_id'] = transcript_cred_offer['cred_def_id']
alice['transcript_cred_offer'] = faber['transcript_cred_offer']
print(transcript_cred_offer)

## 7.4 [Alice -> Faber] Credential Request 생성 및 전달

In [ ]:
print("\"Alice\" -> Create and store \"Alice\" Master Secret in Wallet")
alice['master_secret_id'] = await anoncreds.prover_create_master_secret(alice['wallet'], None)

print("\"Alice\" -> Get \"Faber Transcript\" Credential Definition from Ledger")
(alice['faber_transcript_cred_def_id'], alice['faber_transcript_cred_def']) = \
    await get_cred_def(alice['pool'], 
                       alice_did_info['did'], 
                       transcript_cred_offer['cred_def_id'])

print("\"Alice\" -> Create \"Transcript\" Credential Request for Faber")
(alice['transcript_cred_request'], alice['transcript_cred_request_metadata']) = \
    await anoncreds.prover_create_credential_req(alice['wallet'], 
                                                 alice_did_info['did'], 
                                                 alice['transcript_cred_offer'], 
                                                 alice['faber_transcript_cred_def'], 
                                                 alice['master_secret_id'])

print("\"Alice\" -> Send \"Transcript\" Credential Request to Faber")
alice['transcript_cred_values'] = json.dumps({
	"first_name": {"raw": "Alice", "encoded": "1139481716457488690172217916278103335"},
	"last_name": {"raw": "Garcia", "encoded": "5321642780241790123587902456789123452"},
	"degree": {"raw": "Bachelor of Science, Marketing", "encoded": "12434523576212321"},
	"status": {"raw": "graduated", "encoded": "2213454313412354"},
	"ssn": {"raw": "123-45-6789", "encoded": "3124141231422543541"},
	"year": {"raw": "2015", "encoded": "2015"},
	"average": {"raw": "5", "encoded": "5"}
})

faber['alice_transcript_cred_values'] = alice['transcript_cred_values']
faber['transcript_cred_request'] = alice['transcript_cred_request']

## 7.5 [Faber -> Alice] Credential 생성 및 전달

In [ ]:
print("\"Faber\" -> Create \"Transcript\" Credential for Alice")

faber['transcript_cred'], _, _ = await anoncreds.issuer_create_credential(faber['wallet'], faber['transcript_cred_offer'],
                                                                          faber['transcript_cred_request'],
                                                                          faber['alice_transcript_cred_values'], None, None)

print("\"Faber\" -> Send  \"Transcript\" Credential to Alice")
alice['transcript_cred'] = faber['transcript_cred']

## 7.6 [Alice] Credential(Transcript) 저장(in Wallet)

In [ ]:
print("\"Alice\" -> Store \"Transcript\" Credential from Faber")
_, alice['transcript_cred_def'] = await get_cred_def(alice['pool'], alice_did_info['did'],
                                                     alice['transcript_cred_def_id'])

await anoncreds.prover_store_credential(alice['wallet'], None, alice['transcript_cred_request_metadata'],
                                        alice['transcript_cred'], alice['transcript_cred_def'], None)

# 8. Acme Corp 입사 지원(for Alice)

## 8.1 [Acme -> Alice] Job-Application Proof Request 생성 및 전달

In [ ]:
print("==============================")
print("== Apply for the job with Acme - Transcript proving ==")
print("------------------------------")

print("\"Acme\" -> Create \"Job-Application\" Proof Request")
nonce = await anoncreds.generate_nonce()
acme['job_application_proof_request'] = json.dumps({
    'nonce': nonce,
    'name': 'Job-Application',
    'version': '0.1',
    'requested_attributes': {
        'attr1_referent': {
            'name': 'first_name'
        },
        'attr2_referent': {
            'name': 'last_name'
        },
        'attr3_referent': {
            'name': 'degree',
            'restrictions': [{'cred_def_id': faber['transcript_cred_def_id']}]
        },
        'attr4_referent': {
            'name': 'status',
            'restrictions': [{'cred_def_id': faber['transcript_cred_def_id']}]
        },
        'attr5_referent': {
            'name': 'ssn',
            'restrictions': [{'cred_def_id': faber['transcript_cred_def_id']}]
        },
        'attr6_referent': {
            'name': 'phone_number'
        }
    },
    'requested_predicates': {
        'predicate1_referent': {
            'name': 'average',
            'p_type': '>=',
            'p_value': 4,
            'restrictions': [{'cred_def_id': faber['transcript_cred_def_id']}]
        }
    }
})

print("\"Acme\" -> Get key for Alice did")
acme['alice_key_for_acme'] = \
        await did.key_for_did(acme['pool'], acme['wallet'], alice_did_info['did'])

print("\"Acme\" -> Send \"Job-Application\" Proof Request to Alice")
alice['job_application_proof_request'] = acme['job_application_proof_request']

## 8.2 [Alice -> Acme] Job-Application Proof  생성 및 전달

In [ ]:
print("\"Alice\" -> Get credentials for \"Job-Application\" Proof Request")
print(">> alice['job_application_proof_request'] : ")
print(alice['job_application_proof_request'])
search_for_job_application_proof_request = \
    await anoncreds.prover_search_credentials_for_proof_req(alice['wallet'],
                                                            alice['job_application_proof_request'], None)

cred_for_attr1 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr1_referent')
print(">> cred_for_attr1 : ")
print(cred_for_attr1)
cred_for_attr2 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr2_referent')
cred_for_attr3 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr3_referent')
cred_for_attr4 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr4_referent')
cred_for_attr5 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr5_referent')
cred_for_predicate1 = \
    await get_credential_for_referent(search_for_job_application_proof_request, 'predicate1_referent')

await anoncreds.prover_close_credentials_search_for_proof_req(search_for_job_application_proof_request)

alice['creds_for_job_application_proof'] = {cred_for_attr1['referent']: cred_for_attr1,
                                            cred_for_attr2['referent']: cred_for_attr2,
                                            cred_for_attr3['referent']: cred_for_attr3,
                                            cred_for_attr4['referent']: cred_for_attr4,
                                            cred_for_attr5['referent']: cred_for_attr5,
                                            cred_for_predicate1['referent']: cred_for_predicate1}

alice['schemas'], alice['cred_defs'], alice['revoc_states'] = \
    await prover_get_entities_from_ledger(alice['pool'], alice_did_info['did'],
                                          alice['creds_for_job_application_proof'], alice['name'])

print("\"Alice\" -> Create \"Job-Application\" Proof")
alice['job_application_requested_creds'] = json.dumps({
    'self_attested_attributes': {
        'attr1_referent': 'Alice',
        'attr2_referent': 'Garcia',
        'attr6_referent': '123-45-6789'
    },
    'requested_attributes': {
        'attr3_referent': {'cred_id': cred_for_attr3['referent'], 'revealed': True},
        'attr4_referent': {'cred_id': cred_for_attr4['referent'], 'revealed': True},
        'attr5_referent': {'cred_id': cred_for_attr5['referent'], 'revealed': True},
    },
    'requested_predicates': {'predicate1_referent': {'cred_id': cred_for_predicate1['referent']}}
})

alice['job_application_proof'] = \
    await anoncreds.prover_create_proof(alice['wallet'], alice['job_application_proof_request'],
                                        alice['job_application_requested_creds'], alice['master_secret_id'],
                                        alice['schemas'], alice['cred_defs'], alice['revoc_states'])

print("\"Alice\" -> Send \"Job-Application\" Proof to Acme")
acme['job_application_proof'] = alice['job_application_proof']

## 8.3 [Acme] Job-Application Proof 검증

In [ ]:
job_application_proof = json.loads(acme['job_application_proof'])
acme['schemas'], acme['cred_defs'], acme['revoc_ref_defs'], acme['revoc_regs'] = \
    await verifier_get_entities_from_ledger(acme['pool'], acme['did'],
                                            job_application_proof['identifiers'], acme['name'])

print("\"Acme\" -> Verify \"Job-Application\" Proof from Alice")
assert 'Bachelor of Science, Marketing' == \
       job_application_proof['requested_proof']['revealed_attrs']['attr3_referent']['raw']
assert 'graduated' == \
       job_application_proof['requested_proof']['revealed_attrs']['attr4_referent']['raw']
assert '123-45-6789' == \
       job_application_proof['requested_proof']['revealed_attrs']['attr5_referent']['raw']

assert 'Alice' == job_application_proof['requested_proof']['self_attested_attrs']['attr1_referent']
assert 'Garcia' == job_application_proof['requested_proof']['self_attested_attrs']['attr2_referent']
assert '123-45-6789' == job_application_proof['requested_proof']['self_attested_attrs']['attr6_referent']

assert await anoncreds.verifier_verify_proof(acme['job_application_proof_request'], acme['job_application_proof'],
                                             acme['schemas'], acme['cred_defs'], acme['revoc_ref_defs'],
                                             acme['revoc_regs'])

# 9. 종료

In [ ]:
print("==============================")

print("\"Sovrin Steward\" -> Close and Delete wallet")
await wallet.close_wallet(steward['wallet'])
await wallet.delete_wallet(steward['wallet_config'], steward['wallet_credentials'])

print("\"Government\" -> Close and Delete wallet")
await wallet.close_wallet(government['wallet'])
await wallet.delete_wallet(government['wallet_config'], government['wallet_credentials'])

print("\"Faber\" -> Close and Delete wallet")
await wallet.close_wallet(faber['wallet'])
await wallet.delete_wallet(faber['wallet_config'], faber['wallet_credentials'])

print("\"Acme\" -> Close and Delete wallet")
await wallet.close_wallet(acme['wallet'])
await wallet.delete_wallet(acme['wallet_config'], acme['wallet_credentials'])

print("\"Thrift\" -> Close and Delete wallet")
await wallet.close_wallet(thrift['wallet'])
await wallet.delete_wallet(thrift['wallet_config'], thrift['wallet_credentials'])

print("\"Alice\" -> Close and Delete wallet")
await wallet.close_wallet(alice['wallet'])
await wallet.delete_wallet(alice['wallet_config'], alice['wallet_credentials'])

print("Close and Delete pool")
await pool.close_pool_ledger(pool_['handle'])
await pool.delete_pool_ledger_config(pool_['name'])

print("Getting started -> done")

### ※ 참고 문서 - https://medium.com/@kctheservant/exploring-hyperledger-indy-through-indy-dev-example-10075d2547ae